# **GBS data processing workflow**

**Michel Moser, PhD, Institute of Plant Sciences**

This notebook contains step-by-step explanaitions from Illumina read-processing to marker creation to be further used in genetic map construction. 

#### **Dependecies**

>  [**fastq-mcf**](http://expressionanalysis.github.io/ea-utils/)  - skewing detection and quality filtering of Illumina reads    

>  [**bowtie2**](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml)   - fast and sensitive short-read aligner    

> [**samtools**](https://github.com/samtools/samtools/releases) - manipulating tool for next-generation sequencing data 

> [**LepMAP3**](https://sourceforge.net/projects/lep-map3/) - linkage map construction suite



### **Acknowledgements**

This collection of tools are not my own work and should be cited respectively when used.  


In [ ]:
#### 1. quality filter (QC) of raw demultiplexed reads   


In [7]:
%%bash 
fastq-mcf -q 30 -l 50

bash: line 1: fastq-mcf: command not found


In [3]:



### 2. align QC reads to genome   
$bowtie2 --very-sensitive --no-unal --rg-id F7_RIL103 --rg SM:blubb --rg LB:blubb --rg PI:blubby --rg PL:ILLUMINA

### 3. prepare fofn's for posterior calls and LM3
        genetic maps using parts of LM3 scripts, downloadable at (https://sourceforge.net/projects/lep-map3/)

        bam_list.txt: space delimited fofn to bam-files
         example: 
         ../F7_plate1.95.b2.peex113.s.bam ../F7_plate1.94.b2.peex113.s.bam ../F7_plate1.1.b2.peex113.s.bam ../F7_plate1.10.b2.peex113.s.bam ../F7_plate1.11.b2.peex113.s.bam 
         
         mapping.txt: space delimited file of sample names in same order as bam_list.txt
         example: 
         axillaris axillaris exserta 1 2 3 4                   

* 3.1: create posterior basecalls
          $samtools mpileup -q 10 -Q 10 -s $(cat bam_list.txt) |awk -f LEP-MAP3/scripts/pileupParser2.awk |awk -f LEP-MAP3/scripts/pileup2posterior.awk |head -1000 |gzip > tt.post.gz             

        pedigree.txt: tab-delimited file of relation between individuals
        resemble F7 as F2: 
        
        #family  indID   mother   father   sex(1=female;2=male)  phenotype                       
        F2      exserta 0       0       1       0   
        F2      axillaris       0       0       2       0                                        
        F2      P1      exserta axillaris       1       0                                        
        F2      P2      exserta axillaris       2       0                                        
        F2      1       P1      P2      0       0   
        F2      2       P1      P2      0       0   
        F2      3       P1      P2      0       0   
        F2      4       P1      P2      0       0   


* 3.2: transform pedigree.txt

        $LEP-MAP3/scripts/transpose_tab Michel.F4.ped |awk '{print "CHR\tPOS\t"$0}' > Michel.F4.tped 
               
* 3.3: create posterior calls with pedigree and posterior calls    

         $zcat tt.post.gz |java -cp LEP-MAP3/bin/ ParentCall2 data=Michel.F2.tped posteriorFile=- removeNonInformative=1  > tt.postcall           
         
         
         
         
## 4. Transfer and filter posterior calls to csv (to use as input in R/ASmap) 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [145]:
import pandas as pd
pd.get_option("display.max_rows")
pd.set_option("display.max_rows",8)
pd.set_option("display.max_columns",10)




test_K = pd.read_csv("/media/mmoser/data1/P.EXSERTA/PEXV1.1.3/genetic_maps/F7_total.PD.postcall", sep = "\t")
test_K.columns

#filter out dummy parents
cols = test_K.columns.str.contains("RIL|axillaris|exserta|POS|CHR") 
cols
cols = test_K.columns[cols]
test_Kril = test_K[cols].copy()

for i in cols[2:]: 
    #print(i)
    test_Kril[i] = test_K[i].apply(lambda x: x.split(' '))
test_Kril.columns

Index(['CHR', 'POS', 'exserta', 'axillaris', 'RIL_196', 'RIL_197', 'RIL_198',
       'RIL_199', 'RIL_200', 'RIL_201',
       ...
       'RIL_187', 'RIL_105', 'RIL_188', 'RIL_189', 'RIL_190', 'RIL_191',
       'RIL_192', 'RIL_194', 'RIL_195', 'RIL_106'],
      dtype='object', length=199)

In [146]:
test_Kril.loc[1,["axillaris", "exserta" ]]
test_Kril.POS = test_Kril.POS.apply(str)
test_Kril['marker'] = test_Kril[['CHR', 'POS']].apply(lambda x: '_'.join(x), axis=1)
display(test_Kril)

,CHR,POS,exserta,axillaris,RIL_196,...,RIL_192,RIL_194,RIL_195,RIL_106,marker
0,Peex113Ctg00004,119009,"[6.59054E-6, 1.98058E-13, 1.0, 1.98058E-13, 0,...","[0, 0, 7.61952E-9, 0, 0, 7.61952E-9, 0, 1.0, 7...","[1.0, 0, 1.226418E-4, 0, 0, 0, 0, 0, 0, 0]",...,"[1.71054E-35, 0, 0.001959652, 0, 0, 0, 0, 1.0,...","[1.0, 0, 0.0626044, 0, 0, 0, 0, 4.13586E-18, 0...","[1.0, 0, 0.01566154, 0, 0, 0, 0, 4.60461E-25, ...","[1.0, 0, 0.125167, 0, 0, 0, 0, 1.23952E-14, 0, 0]",Peex113Ctg00004_119009
1,Peex113Ctg00043,151704,"[0, 0, 1.1566E-13, 0, 0, 0.583969, 0, 1.0, 1.1...","[0, 9.73024E-7, 0, 0, 1.0, 9.73024E-7, 9.73024...","[0, 0, 0, 0, 0, 7.67806E-6, 0, 1.0, 0, 0]",...,"[0, 0, 0, 0, 1.0, 0.01566154, 0, 4.60461E-25, ...","[0, 0, 0, 0, 1.0, 2.45376E-4, 0, 0, 0, 0]","[0, 0, 0, 0, 1.0, 0.01566154, 0, 4.60461E-25, ...","[0, 0, 0, 0, 1.0, 1.534558E-5, 0, 0, 0, 0]",Peex113Ctg00043_151704
2,Peex113Ctg00050,84548,"[0, 0, 1.38841E-20, 0, 0, 1.38841E-20, 0, 1.0,...","[0, 0, 0, 1.3864E-20, 0, 0, 1.3864E-20, 0, 1.3...","[0, 0, 0, 0, 0, 0, 0, 1.0, 2.45288E-4, 0]",...,"[0, 0, 0, 0, 0, 0, 0, 0, 1.22863E-4, 1.0]","[0, 0, 0, 0, 0, 0, 0, 3.71484E-11, 0.25025, 1.0]","[0, 0, 0, 0, 0, 0, 0, 0, 2.39252E-10, 1.0]","[0, 0, 0, 0, 0, 0, 0, 3.71484E-11, 0.25025, 1.0]",Peex113Ctg00050_84548
3,Peex113Ctg00050,84557,"[0, 0, 1.3904E-20, 0, 0, 1.3904E-20, 0, 1.0, 1...","[1.0, 1.20089E-19, 1.20089E-19, 1.20089E-19, 0...","[5.12648E-32, 0, 0.003918, 0, 0, 0, 0, 1.0, 0, 0]",...,"[1.0, 0, 1.22908E-4, 0, 0, 0, 0, 0, 0, 0]","[1.0, 0, 0.250338, 0, 0, 0, 0, 7.65411E-11, 0, 0]","[1.0, 0, 2.41356E-10, 0, 0, 0, 0, 0, 0, 0]","[1.0, 0, 0.25025, 0, 0, 0, 0, 3.71484E-11, 0, 0]",Peex113Ctg00050_84557
...,...,...,...,...,...,...,...,...,...,...,...
1939,Peex113Ctg04638,55082,"[0, 0, 1.73535E-21, 0, 0, 1.73535E-21, 0, 1.0,...","[1.0, 3.09949E-5, 3.09949E-5, 3.09949E-5, 0, 0...","[5.7075E-39, 0, 9.80152E-4, 0, 0, 0, 0, 1.0, 0...",...,"[0, 0, 2.45202E-4, 0, 0, 0, 0, 1.0, 0, 0]","[0, 0, 1.229064E-4, 0, 0, 0, 0, 1.0, 0, 0]","[0, 0, 2.88714E-14, 0, 0, 0, 0, 1.0, 0, 0]","[4.60461E-25, 0, 0.01566154, 0, 0, 0, 0, 1.0, ...",Peex113Ctg04638_55082
1940,Peex113Ctg04867,106970,"[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]","[1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]",Peex113Ctg04867_106970
1941,Peex113Ctg00483,38002,"[0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0]","[0, 0, 0, 0, 0, 6.43918E-33, 0, 1.0, 0, 0]",...,"[0, 0, 0, 0, 1.0, 9.00004E-19, 0, 0, 0, 0]","[0, 0, 0, 0, 1.0, 7.52582E-9, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 2.72236E-26, 0, 1.0, 0, 0]","[0, 0, 0, 0, 9.0003E-40, 1.0, 0, 0, 0, 0]",Peex113Ctg00483_38002
1942,Peex113Ctg00483,38010,"[0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]","[0, 0, 0, 0, 0, 0, 0, 0, 1.630864E-33, 1.0]",...,"[0, 0, 0, 0, 0, 0, 0, 1.0, 8.85506E-19, 0]","[0, 0, 0, 0, 0, 0, 0, 1.0, 9.42004E-10, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1.338332E-26, 1.0]","[0, 0, 0, 0, 0, 0, 0, 1.200865E-42, 1.0, 0]",Peex113Ctg00483_38010


In [151]:
mapping = [ "AA", "AC", "AG","AT", "CC", "CG", "CT", "GG", "GT" ,"TT"]
print(test_Kril.shape)
tt = test_Kril[cols[2:]].copy()
tt1 = test_Kril.copy()
c = 0
cc = 0
for key, value in tt.iteritems(): 
    cc += 1
    if cc % 10 == 0: 
        print("{} individuals processed ...".format(cc))
    for t, i in enumerate(value): 
        c += 1
        if i.count('1.0') > 1: 
            g = "NA"
        else: 
            g = mapping[i.index('1.0')]
        tt1[key].iloc[t] = g

(1943, 200)
10 individuals processed ...
20 individuals processed ...
30 individuals processed ...
40 individuals processed ...
50 individuals processed ...
60 individuals processed ...
70 individuals processed ...
80 individuals processed ...
90 individuals processed ...
100 individuals processed ...
110 individuals processed ...
120 individuals processed ...
130 individuals processed ...
140 individuals processed ...
150 individuals processed ...
160 individuals processed ...
170 individuals processed ...
180 individuals processed ...
190 individuals processed ...


In [152]:
display(tt1)

,CHR,POS,exserta,axillaris,RIL_196,...,RIL_192,RIL_194,RIL_195,RIL_106,marker
0,Peex113Ctg00004,119009,AG,GG,AA,...,GG,AA,AA,AA,Peex113Ctg00004_119009
1,Peex113Ctg00043,151704,GG,CC,GG,...,CC,CC,CC,CC,Peex113Ctg00043_151704
2,Peex113Ctg00050,84548,GG,TT,GG,...,TT,TT,TT,TT,Peex113Ctg00050_84548
3,Peex113Ctg00050,84557,GG,AA,GG,...,AA,AA,AA,AA,Peex113Ctg00050_84557
...,...,...,...,...,...,...,...,...,...,...,...
1939,Peex113Ctg04638,55082,GG,AA,GG,...,GG,GG,GG,GG,Peex113Ctg04638_55082
1940,Peex113Ctg04867,106970,CC,AA,CC,...,CC,CC,CC,CC,Peex113Ctg04867_106970
1941,Peex113Ctg00483,38002,CC,GG,GG,...,CC,CC,GG,CG,Peex113Ctg00483_38002
1942,Peex113Ctg00483,38010,GG,TT,TT,...,GG,GG,TT,GT,Peex113Ctg00483_38010


## transfer basecalls to genotypes using parental columns as reference

In [283]:
%%time

#genotype the individuals according to parents
out = tt1.copy()
c = 0
mis_count = 0
identical = 0
#print(tt1.shape)

for k, value in tt1.iterrows():
    c += 1
    exs = value['exserta']
    axs = value['axillaris']
    if c % 100 == 0: 
        print(c)
    
    #IF partental basecalls are not homozygous
    if len(set(exs)) + len(set(axs)) > 2: 
        #problematic markers
        #fixed_marker = handle_marker(value)
        #out.iloc[k, :] = fixed_marker
        mis_count += 1
        
    
    elif set(exs) == set(axs): 
        identical += 1
    
    
    #IF partental basecalls are homozygous
    else:
        exsB = list(set(exs))[0]
        axsB = list(set(axs))[0]
        het1 = axsB + exsB
        het2 = exsB + axsB
        
        
        for individual, i in value[4:tt1.shape[1]].iteritems(): 
            if i == axs: 
                out.at[k, individual] = "AX"
            elif i == exs: 
                out.at[k, individual] = "EX"
            elif i == het1 or i == het2: 
                out.at[k, individual] = "HET"
            else : 
                out.at[k, individual] = "NA"
        

print("{} markers not being homozygous for the parents".format(mis_count))        
print("{} markers identical between parental species".format(identical))        


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
86 markers not being homozygous for the parents
5 markers identical between parental species
CPU times: user 5.68 s, sys: 7.98 ms, total: 5.69 s
Wall time: 5.69 s


In [284]:
display(out)

print(out.shape, tt1.shape)
tt1.to_csv("/media/mmoser/data1/P.EXSERTA/PEXV1.1.3/genetic_maps/markers/F7-K/F7-K.bases.csv", index = False)
out.to_csv("/media/mmoser/data1/P.EXSERTA/PEXV1.1.3/genetic_maps/markers/F7-K/F7-K.geno.csv", index = False)

,CHR,POS,exserta,axillaris,RIL_196,...,RIL_192,RIL_194,RIL_195,RIL_106,marker
0,Peex113Ctg00004,119009,AG,GG,AA,...,GG,AA,AA,AA,Peex113Ctg00004_119009
1,Peex113Ctg00043,151704,GG,CC,EX,...,AX,AX,AX,AX,NA
2,Peex113Ctg00050,84548,GG,TT,EX,...,AX,AX,AX,AX,NA
3,Peex113Ctg00050,84557,GG,AA,EX,...,AX,AX,AX,AX,NA
...,...,...,...,...,...,...,...,...,...,...,...
1939,Peex113Ctg04638,55082,GG,AA,EX,...,EX,EX,EX,EX,NA
1940,Peex113Ctg04867,106970,CC,AA,EX,...,EX,EX,EX,EX,NA
1941,Peex113Ctg00483,38002,CC,GG,AX,...,EX,EX,AX,HET,NA
1942,Peex113Ctg00483,38010,GG,TT,AX,...,EX,EX,AX,HET,NA


(1943, 200) (1943, 200)


## Filter HQ markers and count number of genotpyes per marker

In [285]:
out.index = tt1.marker
row["exserta"]

'GG'

In [286]:
co = 0

marker_l = []

#iterate over markers
for index, row in out.iloc[:,2:out.shape[1]-1].iterrows():
    t = row.value_counts().sort_index()
    #print(t)
    
    #filter out heterzygous parents and identical parents
    if "AX" in t.index and row["exserta"] != row["axillaris"]:
        
        #replace heterzygous sites with NA
        if "HET" in t.index: 
            #print(row)
            row1 = row.replace("HET", "NA")
            marker_l.append(row1)
        else: 
            marker_l.append(row)


marker_df = pd.concat(marker_l, axis=1, keys=[s for s in out.index])
            


#iterate over individuals and summarise marker appearance

out_list = []
header = ["individual", "AX", "EX", "NA", "tot"]

for index, individual_s in marker_df.iterrows(): 
    
    t = individual_s.value_counts().sort_index()
    
    if "AX" not in t.index: 
        continue
    #print(column, [val for pair in zip(t.tolist(), t.index.tolist()) for val in pair])
    out_list.append([index, t["AX"], t["EX"], t["NA"], t["AX"]+ t["EX"]+ t["NA"]])


#iterate over markers and summarise marker appearance

out_list2 = []
header2 = ["marker", "AX", "EX", "NA", "tot"]

marker_dfT = marker_df.T

for index, marker_s in marker_dfT.iterrows(): 
    
    t = marker_s.value_counts().sort_index()
    
    if "AX" not in t.index: 
        print(marker_s)
        out_list2.append([index, "-", t["EX"], t["NA"], t["EX"]+ t["NA"]])
        continue
    elif "EX" not in t.index:
        print(marker_s)
        out_list2.append([index, t["AX"], "-", t["NA"], t["AX"]+  t["NA"]])
        continue
    
    #print(column, [val for pair in zip(t.tolist(), t.index.tolist()) for val in pair])
    out_list2.append([index, t["AX"], t["EX"], t["NA"], t["AX"]+ t["EX"]+ t["NA"]])


    
    
    
#harvest list

df1 = pd.DataFrame(out_list, columns=header)
df2 = pd.DataFrame(out_list2, columns=header2)

display(marker_df)


marker_df.to_csv("/media/mmoser/data1/P.EXSERTA/PEXV1.1.3/genetic_maps/markers/F7-K/LM3_F7_K.markers.csv", index = True)
df1.to_csv("/media/mmoser/data1/P.EXSERTA/PEXV1.1.3/genetic_maps/markers/F7-K/LM3_F7_K.INDIVIDUAL.genoSUMMARY.csv", index = False)
df2.to_csv("/media/mmoser/data1/P.EXSERTA/PEXV1.1.3/genetic_maps/markers/F7-K/LM3_F7_K.MARKER.genoSUMMARY.csv", index = False)

,Peex113Ctg00004_119009,Peex113Ctg00043_151704,Peex113Ctg00050_84548,Peex113Ctg00050_84557,Peex113Ctg00050_84666,...,Peex113Ctg02378_57283,Peex113Ctg02421_63455,Peex113Ctg02421_63493,Peex113Ctg02437_62689,Peex113Ctg00240_55371
exserta,GG,GG,GG,CC,CC,...,AA,GG,CC,CC,GG
axillaris,CC,TT,AA,TT,GG,...,GG,AA,AA,GG,TT
RIL_196,EX,EX,EX,EX,EX,...,EX,EX,EX,AX,AX
RIL_197,AX,AX,AX,AX,AX,...,NA,EX,EX,AX,AX
...,...,...,...,...,...,...,...,...,...,...,...
RIL_192,AX,AX,AX,AX,EX,...,AX,EX,EX,EX,EX
RIL_194,AX,AX,AX,AX,AX,...,AX,EX,EX,EX,EX
RIL_195,AX,AX,AX,AX,AX,...,EX,EX,EX,AX,AX
RIL_106,AX,AX,AX,AX,EX,...,AX,EX,EX,NA,NA


### further filtering of individuals and markers will follow in R